In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np 
from torch.utils.data import DataLoader, Dataset

In [24]:
# import dataset using pd 
# temporary small dataset
english_to_french = [

    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")

]

all_text = ''.join([en + "<sep>" + fr + " " for en, fr in english_to_french])
all_text

"I am cold<sep>J'ai froid You are tired<sep>Tu es fatigué He is hungry<sep>Il a faim She is happy<sep>Elle est heureuse We are friends<sep>Nous sommes amis They are students<sep>Ils sont étudiants The cat is sleeping<sep>Le chat dort The sun is shining<sep>Le soleil brille We love music<sep>Nous aimons la musique She speaks French fluently<sep>Elle parle français couramment He enjoys reading books<sep>Il aime lire des livres They play soccer every weekend<sep>Ils jouent au football chaque week-end The movie starts at 7 PM<sep>Le film commence à 19 heures She wears a red dress<sep>Elle porte une robe rouge We cook dinner together<sep>Nous cuisinons le dîner ensemble He drives a blue car<sep>Il conduit une voiture bleue They visit museums often<sep>Ils visitent souvent des musées The restaurant serves delicious food<sep>Le restaurant sert une délicieuse cuisine She studies mathematics at university<sep>Elle étudie les mathématiques à l'université We watch movies on Fridays<sep>Nous regar

In [17]:
# tokenizer: build a list of tokens in each language. BPE utilized here. 
test = "hello, world. this is a test of the byte pair encoding algorithm.".encode("utf-8")
encoded = list(map(int, test))

# bpe hyperparameters: 
num_merges = 20
replacements = {}  # order of merges matters

def merge(ids, pair, idx): 
  new_ids = []
  i = 0
  while i < len(ids): 
    if i + 1 < len(ids) and ids[i] == pair[0] and ids[i + 1] == pair[1]: 
      new_ids.append(idx)
      i += 2
    else: 
      new_ids.append(ids[i])
      i += 1
  return new_ids

# find most common pair of bytes, and replace with new byte
def get_top_pair(text): 
  counts = {pair: counts.get(pair,0)+1 for pair in zip(text, text[1:])}
  return counts

def bpe_encode():
  for i in range(num_merges): 
    stats = get_top_pair(encoded)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    ids = merge(encoded, pair, idx)
    replacements[pair] = idx

def bpe_decode(): 
  pass


{(104, 101): 2,
 (101, 108): 1,
 (108, 108): 1,
 (108, 111): 1,
 (111, 44): 1,
 (44, 32): 1,
 (32, 119): 1,
 (119, 111): 1,
 (111, 114): 2,
 (114, 108): 1,
 (108, 100): 1,
 (100, 46): 1,
 (46, 32): 1,
 (32, 116): 3,
 (116, 104): 3,
 (104, 105): 1,
 (105, 115): 2,
 (115, 32): 2,
 (32, 105): 1,
 (32, 97): 2,
 (97, 32): 1,
 (116, 101): 2,
 (101, 115): 1,
 (115, 116): 1,
 (116, 32): 1,
 (32, 111): 1,
 (111, 102): 1,
 (102, 32): 1,
 (101, 32): 2,
 (32, 98): 1,
 (98, 121): 1,
 (121, 116): 1,
 (32, 112): 1,
 (112, 97): 1,
 (97, 105): 1,
 (105, 114): 1,
 (114, 32): 1,
 (32, 101): 1,
 (101, 110): 1,
 (110, 99): 1,
 (99, 111): 1,
 (111, 100): 1,
 (100, 105): 1,
 (105, 110): 1,
 (110, 103): 1,
 (103, 32): 1,
 (97, 108): 1,
 (108, 103): 1,
 (103, 111): 1,
 (114, 105): 1,
 (105, 116): 1,
 (104, 109): 1,
 (109, 46): 1}

In [ ]:
# encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hidden_dim)

    def forward(self, src):
        # src: [src len, batch size]
        embedded = self.embedding(src)  # embedded: [src len, batch size, emb dim]
        outputs, hidden = self.gru(embedded)  # hidden: [1, batch size, hidden dim]
        return hidden
        

In [ ]:
# decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden):
        # input: [batch size]
        # hidden: [1, batch size, hidden dim]
        input = input.unsqueeze(0)  # input: [1, batch size]
        embedded = self.embedding(input)  # embedded: [1, batch size, emb dim]
        output, hidden = self.gru(embedded, hidden)
        prediction = self.fc(output.squeeze(0))  # prediction: [batch size, output dim]
        return prediction, hidden


In [ ]:
# combined model from encoder and decoder
class Translator(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Translator, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [src len, batch size]
        # trg: [trg len, batch size]
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(src)
        
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        
        return outputs
